In [137]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


In [138]:
df=pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [139]:
#preprocessing the data
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [140]:
#Encode categhorical variable
label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])

In [141]:
#Encode OneHot 
encoder_geo=OneHotEncoder(handle_unknown='ignore')
encoded=encoder_geo.fit_transform(df[['Geography']]).toarray()
encoded_geo=pd.DataFrame(encoded,columns=encoder_geo.get_feature_names_out(['Geography']))

In [142]:
df=pd.concat([df.drop('Geography',axis=1),encoded_geo],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [143]:
#split the data into features and targget
x=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

In [144]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)
#standard Scaler
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
x_train=std_scaler.fit_transform(x_train)
x_test=std_scaler.transform(x_test)

In [145]:
#save the encoder and scaler for later use
import pickle
with open('oneHot_encoder.pkl','wb') as file:
    pickle.dump(encoded,file)
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)
with open('scaler.pkl','wb') as file:
    pickle.dump(std_scaler,file)

ANN regression probelm statement

In [146]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

In [147]:
#Building ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1),
])

In [148]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 64)                832       
                                                                 
 dense_24 (Dense)            (None, 32)                2080      
                                                                 
 dense_25 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [149]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.MeanAbsoluteError(name='mean_absolute_error')

In [150]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [151]:
#seeting the tensorboard
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from datetime import datetime
log_dir='regressionlogs/fit/'+datetime.now().strftime('%Y%m%D-%H%M%S')
tenorboard_dallback=TensorBoard(log_dir=log_dir,histogram_freq=1)
earlyStopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [152]:
hostory=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tenorboard_dallback,earlyStopping_callback]

)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 81001.9844 - accuracy: 0.0000e+00 - val_loss: 51184.6523 - val_accuracy: 0.0000e+00
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 50305.4023 - accuracy: 0.0000e+00 - val_loss: 50509.8828 - val_accuracy: 0.0000e+00
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 49860.6562 - accuracy: 0.0000e+00 - val_loss: 50438.6719 - val_accuracy: 0.0000e+00
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 49719.3281 - accuracy: 0.0000e+00 - val_loss: 50387.2461 - val_accuracy: 0.0000e+00
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 49630.4023 - accuracy: 0.0000e+00 - val_loss: 50379.1211 - val_accuracy: 0.0000e+00
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 49505.0352 - accuracy: 0.0000e+00 - val_loss: 50523.3906 - val_accuracy: 0.0000e+00
Epoch 7/100
250/250 [=========================

In [153]:
model.save('regressionModel.h5')

d:\Father's_Document\code\Projects\CustomerChurnPredcition\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [154]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [155]:
%tensorboard --logdir=regressionlogs/fit


Reusing TensorBoard on port 6008 (pid 17688), started 0:09:15 ago. (Use '!kill 17688' to kill it.)

In [156]:
test_loss,test_mae=model.evaluate(x_test,y_test)
print(f"Test MAE {test_mae}")


63/63 [==============================] - 0s 1ms/step - loss: 50365.7031 - accuracy: 0.0000e+00
Test MAE 0.0
